In [1]:
import pandas as pd
import  copy
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 设置显示所有行
pd.set_option('display.max_rows', None)
# 设置显示所有列
pd.set_option('display.max_columns', None)

In [3]:
hr_data = pd.read_excel('最终得分.xlsx', sheet_name='Sheet1')
hr_data = hr_data.set_index('Unnamed: 0').rename_axis(None)

In [4]:
data = hr_data.copy()
for i in hr_data.index.to_list():
    df = data[data.index == i].T.reset_index().rename(columns={"index": "评委"})
    # 增加类别列
    df.loc[0:4, '类别'] = 1
    df.loc[5:16, '类别'] = 2
    df.loc[17:30, '类别'] = 3
    df["类别"] = df["类别"].astype(int)
    # 去除缺失值
    df_cleaned = df.dropna()
    # 剔除两个最大值，分数相同优先剔除类别大的值
    df_cleaned.sort_values(by=[i, "类别"], ascending=[False, False], inplace=True)
    df_trimmed1 = df_cleaned.iloc[2:]
    # 剔除两个最小值，分数相同优先剔除类别大的值
    df_trimmed1.sort_values(by=[i, "类别"], ascending=[False, True], inplace=True)
    df_trimmed2 = df_trimmed1.iloc[:-2]
    # 分组求均值
    group1 = df_trimmed2[(df_trimmed2.index >= 0) & (df_trimmed2.index <= 4)][i].mean()
    group2 = df_trimmed2[(df_trimmed2.index >= 5) & (df_trimmed2.index <= 16)][i].mean()
    group3 = df_trimmed2[(df_trimmed2.index >= 17) & (df_trimmed2.index <= 30)][i].mean()
    group = group1*0.4 + group2*0.4 + group3*0.2
    hr_data.loc[hr_data.index == i, "平均分"] = group
hr_data["平均分"] = hr_data["平均分"].round(2)

In [6]:
hr_data.to_excel('hr_data.xlsx')